# Prospecção de Dados - Data Mining - DI/FCUL 2022/2023

## Lab class TP04a

# Semi Supervised Learning (part I)

*A Semi Supervised Learning Tutorial by Andre Falcao (DI/FCUL 2021-2022)*

### Summary

1. Computing Support and Confidence
2. Identifying Itemsets with Minimum Support
3. Computing Rules

What follows is a simple approach devised for showing the essentials of Semi-supervised learning. What is below is a full working example and we can compute all frequent itemsets in an efficient way, but it is **not** the apriori algorithm


## Computing Support and Confidence


Here a transaction is a set of elements, which should be quite adequate and efficient for pattern matching

Please observe how this data set is structured in Python. It starts as a List of Lists (one element for transaction) which is the transformed in a list of sets. Each set is one transaction. Sets are very optimized for searching and common set operations like Union, Intersection and Subtraction

In [10]:
from time import time, time_ns

In [1]:
import numpy as np
transactions = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]
tr_sets=[set(tr) for tr in transactions]


In [4]:
tr_sets

[{'Eggs', 'Kidney Beans', 'Milk', 'Nutmeg', 'Onion', 'Yogurt'},
 {'Dill', 'Eggs', 'Kidney Beans', 'Nutmeg', 'Onion', 'Yogurt'},
 {'Apple', 'Eggs', 'Kidney Beans', 'Milk'},
 {'Corn', 'Kidney Beans', 'Milk', 'Unicorn', 'Yogurt'},
 {'Corn', 'Eggs', 'Ice cream', 'Kidney Beans', 'Onion'}]

### Processing support

Support is the actual count of occurrences of a set of items in all transactions. A simple way of computing support for any data set modelled as above would be to go for each transaction and verify if the items that we are looking for are a subset

In [11]:
def getSupport(items, Trs):
    t = time_ns()
    N=len(Trs)
    idxs=range(N)
    res=set(idxs)
    for elem in items:
        finds=[idx for idx in idxs if elem in Trs[idx]]
        res = res & set(finds)
    print(time_ns() - t)
    return len(res)/N


Testing the function:

In [12]:
print("Support for Milk:", getSupport(set(["Milk"]), tr_sets))
print("Support for Milk & Eggs:", getSupport(set(["Milk", "Eggs"]), tr_sets))

0
Support for Milk: 0.6
0
Support for Milk & Eggs: 0.4


Actually this function is not particularly efficient (**Discuss why**)

A better approach is using the is subset method of the set data type in Python (Note that we replace the original function)

In [13]:
def getSupport(items, Trs):
    N=len(Trs)
    R=[i for i in range(N) if items.issubset(Trs[i])]
    return len(R)/N

In [14]:
print("Support for Milk:", getSupport(set(["Milk"]), tr_sets))
print("Support for Milk & Eggs:", getSupport(set(["Milk", "Eggs"]), tr_sets))

Support for Milk: 0.6
Support for Milk & Eggs: 0.4


One further function we will need is `getItems()` that, for a list of transactions will return a set with all unique elements presented

In [15]:
def getAllItems(Trs):
    res=set()
    for t in Trs: res=res | t
    return res


In [16]:
print("These are the unique items in the initial set of transactions")
for item in getAllItems(tr_sets):
    print("\t -> ", item)

These are the unique items in the initial set of transactions
	 ->  Corn
	 ->  Onion
	 ->  Apple
	 ->  Yogurt
	 ->  Kidney Beans
	 ->  Dill
	 ->  Eggs
	 ->  Ice cream
	 ->  Nutmeg
	 ->  Unicorn
	 ->  Milk


And we can also compute confidence between item sets (X and Y) very easily using the known equivalence between confidence and support:

$confidence(X\implies Y) = \cfrac{support(X U Y)}{support(X)}$

And using that principle we can compute it in a simple function that receives as arguments X and Y as well as the list of transactions. 

In [17]:
def getConfidence(X, Y, Trs):
    # confidence(X=>Y) = support(X U Y)/support(X)
    supp_X=getSupport(X, Trs)
    supp_XUY=getSupport(X|Y, Trs)
    return supp_XUY/supp_X


In [18]:
conf_me=getConfidence({"Milk"}, {"Eggs"}, tr_sets)
print("Confidence for Milk => Eggs:%6.4f" % conf_me)


Confidence for Milk => Eggs:0.6667


#### Exercise 

Implement two functions: 
* `getLift`
* `getConviction `

that will calculate the Conviction and the Lift

Where

$
Lift(A\implies B) =\frac{Support(A \cup B)}{Support(A)Support(B)}
$


$
Conviction(A\implies B) =\frac{1-Support(B)}{1-Confidence(A\implies B)}
$



In [45]:
#solution
def getLift(X, Y, Trs):
    #do it here!
    supp_XuY = getSupport(X|Y, Trs)
    supp_X = getSupport(X, Trs)
    supp_Y = getSupport(Y, Trs)
    
    try:
        return supp_XuY / supp_X / supp_Y
    except ZeroDivisionError:
        return 0


def getConviction(X, Y, Trs):
    #do it here!
    supp_Y = getSupport(Y, Trs)
    conf_XuY = getConfidence(X, Y, Trs)
    
    try:
        return ( 1 - supp_Y ) / ( 1 - conf_XuY )
    except ZeroDivisionError:
        return np.inf
    

In [29]:
A=['Yogurt','Corn']
B=set(['Milk'])

print("Lift =", getLift(set(A), set(B), tr_sets))
print("Conviction =", getConviction(set(A), set(B), tr_sets))

Lift = 1.6666666666666667
Conviction = 0


Now we can test these functions with the transactions we have

Now let's compute the Statistics associated to the rule ['Yogurt','Corn'] => ['Milk']

In [35]:
def print_rule_statistics(A, B, tr_sets):
    X=set(A)
    Y=set(B)
    supp_x=getSupport(X,tr_sets)
    supp_y=getSupport(Y,tr_sets)
    supp_xy=getSupport(X|Y,tr_sets)
    conf_xy=getConfidence(X,Y, tr_sets)
    print("Support of %s = %6.3f" % (list(X),  supp_x))
    print("Support of %s = %6.3f" % (list(Y),  supp_y))
    print("Support of %s = %6.3f" % (list(X)+list(Y),  supp_xy))
    print("Confidence of %s => %s = %6.3f" % (list(X), list(Y),  conf_xy))
    print(f"Lift of {list(X)} => {list(Y)} = {getLift(X,Y,tr_sets):.3f}")
    print(f"Conviction of {list(X)} => {list(Y)} = {getConviction(X,Y,tr_sets):.3f}")

A=['Yogurt','Corn']
B=set(['Milk'])
print_rule_statistics(A, B, tr_sets)


Support of ['Corn', 'Yogurt'] =  0.200
Support of ['Milk'] =  0.600
Support of ['Corn', 'Yogurt', 'Milk'] =  0.200
Confidence of ['Corn', 'Yogurt'] => ['Milk'] =  1.000
Lift of ['Corn', 'Yogurt'] => ['Milk'] = 1.667
Conviction of ['Corn', 'Yogurt'] => ['Milk'] = 0.000


#### Exercises

* Experiment the rule ['Corn'] => ['Onion'] and it's inverse rule ['Onion'] => ['Corn']. Are the results the same? Should they be?

In [36]:
#solution
print_rule_statistics(["Corn"], ["Onion"], tr_sets)

Support of ['Corn'] =  0.400
Support of ['Onion'] =  0.600
Support of ['Corn', 'Onion'] =  0.200
Confidence of ['Corn'] => ['Onion'] =  0.500
Lift of ['Corn'] => ['Onion'] = 0.833
Conviction of ['Corn'] => ['Onion'] = 0.800


In [37]:
print_rule_statistics(["Onion"], ["Corn"], tr_sets)

Support of ['Onion'] =  0.600
Support of ['Corn'] =  0.400
Support of ['Onion', 'Corn'] =  0.200
Confidence of ['Onion'] => ['Corn'] =  0.333
Lift of ['Onion'] => ['Corn'] = 0.833
Conviction of ['Onion'] => ['Corn'] = 0.900


## Computing minimum support itemsets

As we know, one of the critical aspects in Association Rule Learning is the determination of minimum support itemsets, that is sets of elements that occur with a minimum support level

A brute force approach would check for all possible combinations of unique items, check for their support levels and then output only the ones that are above the minimum support threshold


### A better way to compute minimum support sets

A faster procedure to obtain this is recursive. We start with the items that already have a minimum support level and add other tems that also have minimum support and we proceed until everything is explored and there is nothing to add

A trivial recursive algorithm could be defined this way:

1. select all items (set S) that have support above a threshold ($S_i>t$) 
2. add all items from 1. to a pool of of itemsets with minimum support ($P$)

makeMinSupportSets:
1. Admit set $T_i$ of all items in S except $i$ 
2. verify if itemset $(i|j)$ is not in $P$: if so compute its support and if it is above or equal the threshold ( $S_{i|j}\geq t$ ), and there are still items to add, add ${i|j}$ to $P$, call the function again, removing j from the set S

This procedure can be easily implemented in one function makeMinSupportSets() which is called recursively and a front end function getMinSupport() which is in charge of calling the main function, pre-processing the inputs and return the list of item combinations that satisfy the requirements 


In [38]:
def makeMinSupportSets(cmb, min_supp, its, trs, supports):
    #cmb is the current combination of items
    #min_supp is the minimum support required
    #its is the set of items not yet tested 
    #trs is the list for all transactions
    #supports corresponds to the list of itemsets with support > min_supp (it is actually a dict)
    for it in its-cmb:
        sit={it}
        new_cmb= cmb | sit
        t_new_cmb= tuple(sorted(new_cmb))
        
        if t_new_cmb not in supports: 
            supp=getSupport(new_cmb, trs)
            #print(t_new_cmb, supp)
            
            if supp>=min_supp: 
                supports[t_new_cmb]=supp
                its0=its-sit
                
                if len(its0)>0:
                    makeMinSupportSets(new_cmb, min_supp, its0, trs, supports)

def getMinSupport(trs, min_supp):
    supports={}
    all_items=getAllItems(trs)
    items=[]
    for it in all_items: 
        supp=getSupport({it},trs)
        if supp>min_supp: 
            items.append(it)
    makeMinSupportSets(set(), min_supp, set(items), trs, supports)
    return supports

Let's test the function with a minimum support of 0.4. All the item combinations that occur at or above 40% of the time

In [40]:
sup_sets=getMinSupport(tr_sets, 0.4)
#%timeit supports=getMinSupport(tr_sets, 0.4)

print("Items that occur with support level >= 40%")
sup_list=list(sup_sets)
sup_list.sort()
for s in sup_list:
    print("\t",s, "\tSupport:%6.4f" %sup_sets[s])

Items that occur with support level >= 40%
	 ('Eggs',) 	Support:0.8000
	 ('Eggs', 'Kidney Beans') 	Support:0.8000
	 ('Eggs', 'Kidney Beans', 'Milk') 	Support:0.4000
	 ('Eggs', 'Kidney Beans', 'Onion') 	Support:0.6000
	 ('Eggs', 'Kidney Beans', 'Onion', 'Yogurt') 	Support:0.4000
	 ('Eggs', 'Kidney Beans', 'Yogurt') 	Support:0.4000
	 ('Eggs', 'Milk') 	Support:0.4000
	 ('Eggs', 'Onion') 	Support:0.6000
	 ('Eggs', 'Onion', 'Yogurt') 	Support:0.4000
	 ('Eggs', 'Yogurt') 	Support:0.4000
	 ('Kidney Beans',) 	Support:1.0000
	 ('Kidney Beans', 'Milk') 	Support:0.6000
	 ('Kidney Beans', 'Milk', 'Yogurt') 	Support:0.4000
	 ('Kidney Beans', 'Onion') 	Support:0.6000
	 ('Kidney Beans', 'Onion', 'Yogurt') 	Support:0.4000
	 ('Kidney Beans', 'Yogurt') 	Support:0.6000
	 ('Milk',) 	Support:0.6000
	 ('Milk', 'Yogurt') 	Support:0.4000
	 ('Onion',) 	Support:0.6000
	 ('Onion', 'Yogurt') 	Support:0.4000
	 ('Yogurt',) 	Support:0.6000


#### Exercise
* The above algorithm is **not** the apriori algorithm. In what aspects is it different? What are its potential advantages and disadvantages

#### Advanced Exercises (to do on your own)

* Explain in your own words how `makeMinSupportSets()` works. Identify the recursive part the conditions for its execution and stopping
* Implement a Brute Force approach (`getMinSupportBF()`)


## Making rules


As we want the rules with a minimum confidence level, we just need a function able to process one specific set of items with a minimum support level and get those that have a threshold confidence level

For that a simple function can be written that takes care of it all and receives a frequent itemset to identify possible rules, the set of all existing rules, and a confidence threshold

In [41]:
def checkRules(itemset, Trs, conf_thres):
    rules=[]
    for i in itemset:
        Y={i}
        X=itemset - Y
        if len(X)>0:
            conf=getConfidence(X, Y, Trs)
            if conf>=conf_thres:
                rules.append((X, Y, conf))
    return rules

In [42]:
rules=checkRules(set(('Eggs', 'Kidney Beans', 'Onion', 'Yogurt')), tr_sets, 0.6)

for X,Y,conf in rules:
    print(X, "=>", Y, "\t[confidence: %6.4f]" % conf)

{'Kidney Beans', 'Eggs', 'Yogurt'} => {'Onion'} 	[confidence: 1.0000]
{'Onion', 'Eggs', 'Yogurt'} => {'Kidney Beans'} 	[confidence: 1.0000]
{'Onion', 'Kidney Beans', 'Yogurt'} => {'Eggs'} 	[confidence: 1.0000]
{'Onion', 'Eggs', 'Kidney Beans'} => {'Yogurt'} 	[confidence: 0.6667]


This can be used for all the minimum support itemsets

In [43]:
def getAllRules(MS_itemsets, Trs, conf_level):
    allrules=[]
    for itemset in MS_itemsets:
        R=checkRules(set(itemset), Trs, conf_level)
        allrules+=R
    return allrules

In [44]:
rules= getAllRules(sup_sets, tr_sets, 0.8)
for X,Y,conf in rules:
    print(X, "=>", Y, "\n[confidence: %6.4f]\n" % conf)

{'Onion'} => {'Kidney Beans'} 
[confidence: 1.0000]

{'Onion', 'Eggs'} => {'Kidney Beans'} 
[confidence: 1.0000]

{'Onion', 'Kidney Beans'} => {'Eggs'} 
[confidence: 1.0000]

{'Kidney Beans', 'Eggs', 'Yogurt'} => {'Onion'} 
[confidence: 1.0000]

{'Onion', 'Eggs', 'Yogurt'} => {'Kidney Beans'} 
[confidence: 1.0000]

{'Onion', 'Kidney Beans', 'Yogurt'} => {'Eggs'} 
[confidence: 1.0000]

{'Onion', 'Yogurt'} => {'Kidney Beans'} 
[confidence: 1.0000]

{'Onion'} => {'Eggs'} 
[confidence: 1.0000]

{'Yogurt', 'Eggs'} => {'Onion'} 
[confidence: 1.0000]

{'Onion', 'Yogurt'} => {'Eggs'} 
[confidence: 1.0000]

{'Milk', 'Eggs'} => {'Kidney Beans'} 
[confidence: 1.0000]

{'Milk'} => {'Kidney Beans'} 
[confidence: 1.0000]

{'Milk', 'Yogurt'} => {'Kidney Beans'} 
[confidence: 1.0000]

{'Yogurt'} => {'Kidney Beans'} 
[confidence: 1.0000]

{'Yogurt', 'Eggs'} => {'Kidney Beans'} 
[confidence: 1.0000]

{'Eggs'} => {'Kidney Beans'} 
[confidence: 1.0000]

{'Kidney Beans'} => {'Eggs'} 
[confidence: 0.8000]



#### Question (discuss)
* Do we get all possible rules this way? If not what types of rules are missing?  What can you do about it?

